In [11]:
%pip install gensim

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_25_topics/lda_model_25_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 21: 0.022*"team" + 0.017*"players" + 0.016*"time" + 0.014*"play" + 0.014*"lot" + 0.010*"playing" + 0.010*"coach" + 0.009*"season" + 0.008*"guys" + 0.007*"player"
Topic 16: 0.039*"nba" + 0.019*"draft" + 0.019*"basketball" + 0.017*"league" + 0.014*"pick" + 0.013*"player" + 0.011*"star" + 0.009*"season" + 0.008*"celtics" + 0.008*"lakers"
Topic 8: 0.025*"injury" + 0.019*"season" + 0.016*"injuries" + 0.015*"health" + 0.012*"week" + 0.011*"surgery" + 0.010*"practice" + 0.010*"return" + 0.010*"day" + 0.009*"knee"
Topic 9: 0.048*"coach" + 0.020*"program" + 0.018*"coaches" + 0.017*"season" + 0.016*"coaching" + 0.016*"head_coach" + 0.014*"seasons" + 0.012*"job" + 0.010*"school" + 0.009*"basketball"
Topic 3: 0.022*"st" + 0.020*"tech" + 0.019*"south" + 0.014*"scored" + 0.013*"north" + 0.011*"gamecocks" + 0.009*"win" + 0.009*"round" + 0.009*"mississippi" + 0.008*"acc"
Topic 24: 0.022*"half" + 0.017*"lead" + 0.015*"minutes" + 0.013*"shot" + 0.013*"left" + 0.011*"scored" + 0.010*"play" 

In [15]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [16]:
import json
import pandas as pd

# Assuming your JSON file contains a list of lists, where each inner list is a tokenized document
# Replace '/content/drive/MyDrive/path/to/your/texts.json' with the actual path to your JSON file
texts_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json'

with open(texts_file_path, 'r') as f:
    texts = json.load(f)

print(f"Loaded {len(texts)} documents from {texts_file_path}")

Loaded 149649 documents from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json


In [17]:
# import pandas as pd

# # Load the tokenized data from the Excel file
# try:
#     tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
#     print("Excel file loaded successfully.")
#     # Assuming the original text is in a column named 'text' or similar.
#     # Please adjust the column name if it's different in your Excel file.
#     if 'Main_text_tokenized' in tokenized_data_df.columns:
#         texts = tokenized_data_df['Main_text_tokenized'].tolist()
#         print(f"Extracted {len(texts)} documents.")
#     else:
#         texts = None
#         print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

# except FileNotFoundError:
#     print("Error: The file was not found at the specified path.")
# except Exception as e:
#     print(f"An error occurred while reading the Excel file: {e}")

In [18]:
#TFinding the dominant topic in each sentence
import pandas as pd


def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.3268,"family, school, life, people, home, black, cit...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.2134,"family, school, life, people, home, black, cit...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,16,0.3215,"nba, draft, basketball, league, pick, player, ...","[garden, missed, lonely, nights, season, debut..."
3,3,7,0.182,"olympic, women, world, won, time, united, olym...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.2354,"ncaa, investigation, university, court, statem...","[robinson, talk, rangers, stanley, cup, finals..."
5,5,11,0.3559,"softball, regional, baseball, tbd, harbaugh, i...","[conference, pacific, coach_mike, gillespie, s..."
6,6,12,0.2198,"league, nfl, baseball, season, team, million, ...","[rangers, signed, center, michael, nylander, m..."
7,7,12,0.1571,"league, nfl, baseball, season, team, million, ...","[gavitt, elected, hall, modest, player, dartmo..."
8,8,21,0.2327,"team, players, time, play, lot, playing, coach...","[mythical, element, senior_guard, seimone, aug..."
9,9,4,0.3432,"family, school, life, people, home, black, cit...","[ncaa_women, jim, izard, diana, vines, remembe..."


In [19]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.3268,"family, school, life, people, home, black, cit...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.2134,"family, school, life, people, home, black, cit...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,16,0.3215,"nba, draft, basketball, league, pick, player, ...","[garden, missed, lonely, nights, season, debut..."
3,3,7,0.182,"olympic, women, world, won, time, united, olym...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.2354,"ncaa, investigation, university, court, statem...","[robinson, talk, rangers, stanley, cup, finals..."


In [20]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[51440, 39630, 3498, 118640, 94794, 85634, 16581, 22224, 66098, 110007, 4279, 2260, 71500, 39014, 16008, 40054, 29020, 61097, 100026, 36857]

Topic 1:
[9885, 31667, 22335, 3512, 243, 81958, 23306, 13752, 6144, 40863, 19345, 40013, 117858, 30483, 37017, 60212, 22333, 58390, 15915, 105625]

Topic 2:
[70093, 98680, 93669, 72246, 32693, 120467, 127833, 55312, 80907, 10627, 90129, 86931, 63770, 89542, 131066, 65807, 20809, 86921, 70656, 89525]

Topic 3:
[83656, 94717, 37244, 37334, 105828, 70761, 18918, 8994, 14751, 143432, 111590, 43636, 35554, 63909, 44086, 131802, 148813, 100981, 62250, 43348]

Topic 4:
[63005, 69341, 3633, 23151, 13387, 95419, 25149, 53231, 69979, 60321, 67091, 45848, 50851, 15526, 21668, 15527, 45315, 100772, 89887, 71202]

Topic 5:
[115822, 28071, 92453, 100438, 112277, 49012, 14542, 121819, 83434, 14925, 46933, 124713, 77012, 49104, 94364, 43182, 25870, 91429, 61206, 116620]

Topic 6:
[183

In [21]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,3518
1,2473
2,10361
3,3072
4,5576
5,6502
6,3322
7,9731
8,1274


In [22]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,51440,0,0.8153,"school, college, players, recruiting, basketba...","[school, players, expected, sign, national, le..."
1,39630,0,0.7653,"school, college, players, recruiting, basketba...","[college_basketball, article, tuesday, sports,..."
2,3498,0,0.6688,"school, college, players, recruiting, basketba...","[ahmad, brooks, met, defensive, football_playe..."
3,118640,0,0.6625,"school, college, players, recruiting, basketba...","[richard, pitino, recruiting, picking, gophers..."
4,94794,0,0.6573,"school, college, players, recruiting, basketba...","[anthony, lee, decided, transfer, temple, reds..."
...,...,...,...,...,...
95,84483,0,0.498,"school, college, players, recruiting, basketba...","[ron, darby, awe, walked, campus, spring, poto..."
96,3330,0,0.4978,"school, college, players, recruiting, basketba...","[ncaa, investigate, camp, spokeswoman, ncaa, p..."
97,29648,0,0.4974,"school, college, players, recruiting, basketba...","[nba, minimum, draft, age, affect, prep, schoo..."
98,32481,0,0.4974,"school, college, players, recruiting, basketba...","[father, school, senior, grown, understand, tr..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,9885,1,0.8911,"golf, round, championship, shot, club, tour, t...","[golf, jim, ahern, shot, stroke, lead, round, ..."
101,31667,1,0.8689,"golf, round, championship, shot, club, tour, t...","[trip, masters, rewarding, mesa, resident, mar..."
102,22335,1,0.8632,"golf, round, championship, shot, club, tour, t...","[annika, sorenstam, mad, finished, birdies, gr..."
103,3512,1,0.8401,"golf, round, championship, shot, club, tour, t...","[top, somethings, chasing, tiger, woods, david..."
104,243,1,0.8358,"golf, round, championship, shot, club, tour, t...","[golf, vijay, singh, blew, tiger, woods, rest,..."
...,...,...,...,...,...
195,2151,1,0.7085,"golf, round, championship, shot, club, tour, t...","[laura, diaz, shot, par, yesterday, lead, lpga..."
196,2870,1,0.7081,"golf, round, championship, shot, club, tour, t...","[billy, andrade, shot, consecutive, round, mai..."
197,46429,1,0.7077,"golf, round, championship, shot, club, tour, t...","[stacy, lewis, finished, atop, leaderboard, re..."
198,45336,1,0.7053,"golf, round, championship, shot, club, tour, t...","[dave, shelburne, golf, column, appears, wedne..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2200,18356,22,0.8728,"season, team, guard, player, clark, freshman, ...","[season, sixth, pacific, postseason, advanced,..."
2201,27521,22,0.8458,"season, team, guard, player, clark, freshman, ...","[glance, season, pacific, conference, finish, ..."
2202,11900,22,0.761,"season, team, guard, player, clark, freshman, ...","[polls, accurate, tough, read, sun_devils, pro..."
2203,72259,22,0.7438,"season, team, guard, player, clark, freshman, ...","[measure, women, basketball, progress, win, se..."
2204,13930,22,0.7408,"season, team, guard, player, clark, freshman, ...","[northern, basketball_coaches, worry, recent, ..."
...,...,...,...,...,...
2295,145022,22,0.5869,"season, team, guard, player, clark, freshman, ...","[ranked, ap, top, preseason, poll, straight, t..."
2296,75161,22,0.5865,"season, team, guard, player, clark, freshman, ...","[jared, sullinger, physically, leaner, sulling..."
2297,145013,22,0.5865,"season, team, guard, player, clark, freshman, ...","[missing, ncaa_tournament, time, coach_mike, w..."
2298,37490,22,0.5856,"season, team, guard, player, clark, freshman, ...","[march_madness, record, top, conference, ten, ..."



--- Topic 23 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2300,67634,23,0.88,"boston, globe, eagles, bc, boston_college, pro...","[head_coach, jerry, york, celebrated, boston_c..."
2301,136091,23,0.7599,"boston, globe, eagles, bc, boston_college, pro...","[neil, neil.best, newsday.com, credit, neil, n..."
2302,58995,23,0.5834,"boston, globe, eagles, bc, boston_college, pro...","[ncaa_tournament, sidebar, pdf, chart, data, c..."
2303,77373,23,0.5522,"boston, globe, eagles, bc, boston_college, pro...","[frozen, bc, bu, cornell, tomorrow, p.m, erie,..."
2304,90045,23,0.5237,"boston, globe, eagles, bc, boston_college, pro...","[captured, beanpot, title, competitors, won, b..."
...,...,...,...,...,...
2395,6907,23,0.3278,"boston, globe, eagles, bc, boston_college, pro...","[globe, west, sports, sportswest, marvin, pave..."
2396,88055,23,0.3278,"boston, globe, eagles, bc, boston_college, pro...","[fitting, circumstance, brought, championships..."
2397,84739,23,0.3276,"boston, globe, eagles, bc, boston_college, pro...","[caption, text, illustration, caption, strike,..."
2398,5464,23,0.327,"boston, globe, eagles, bc, boston_college, pro...","[globe, north, sports, notebook, apair, north,..."



--- Topic 24 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2400,33116,24,0.8919,"half, lead, minutes, shot, left, scored, play,...","[final, prayer, bounced, rim, times, falling, ..."
2401,43920,24,0.8645,"half, lead, minutes, shot, left, scored, play,...","[gerry, mcnamara, looked, exhausted, headed, b..."
2402,5628,24,0.8545,"half, lead, minutes, shot, left, scored, play,...","[conference, championships, east_tournament, c..."
2403,43921,24,0.8544,"half, lead, minutes, shot, left, scored, play,...","[gerry, mcnamara, looked, exhausted, headed, b..."
2404,13606,24,0.8534,"half, lead, minutes, shot, left, scored, play,...","[roy_williams, jayhawks, recovered, half, offe..."
...,...,...,...,...,...
2495,105191,24,0.76,"half, lead, minutes, shot, left, scored, play,...","[guard, demetrius, jackson, rough, evening, fo..."
2496,106072,24,0.7599,"half, lead, minutes, shot, left, scored, play,...","[rex, pflueger, tipped, putback, missed, layup..."
2497,82904,24,0.7598,"half, lead, minutes, shot, left, scored, play,...","[minutes, half, ncaa, east_region, semifinal, ..."
2498,137668,24,0.7598,"half, lead, minutes, shot, left, scored, play,...","[city, closing, saturday, afternoon, basketbal..."


In [23]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_25_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_25_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**